In [25]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression

In [26]:
df = pd.read_csv("train.csv")

In [27]:
df.sample(5)

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
868,1728,95.10,ba,b,ai,c,d,af,l,x,...,0,0,1,0,0,0,0,0,0,0
3354,6686,90.60,o,l,ae,f,d,s,d,f,...,0,0,0,0,0,0,0,0,0,0
4171,8336,118.81,x,z,m,c,d,aa,a,c,...,1,0,0,0,0,0,0,0,0,0
3750,7514,98.85,ay,i,as,a,d,v,l,k,...,0,0,1,0,0,0,0,0,0,0
1760,3520,80.34,az,y,b,c,d,ae,j,l,...,0,0,0,0,0,0,1,0,0,0


In [28]:
df = df.drop(['ID'], axis=1)

# Undestanding data

In [29]:
df.shape

(4209, 377)

In [30]:
df.describe()

,y,X10,X11,X12,X13,X14,X15,X16,X17,X18,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
count,4209.000000,4209.000000,4209.0,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,...,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000,4209.000000
mean,100.669318,0.013305,0.0,0.075077,0.057971,0.428130,0.000475,0.002613,0.007603,0.007840,...,0.318841,0.057258,0.314802,0.020670,0.009503,0.008078,0.007603,0.001663,0.000475,0.001426
std,12.679381,0.114590,0.0,0.263547,0.233716,0.494867,0.021796,0.051061,0.086872,0.088208,...,0.466082,0.232363,0.464492,0.142294,0.097033,0.089524,0.086872,0.040752,0.021796,0.037734
min,72.110000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,90.820000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,99.150000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,109.010000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,265.320000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [31]:
df.isnull().sum()

y       0
X0      0
X1      0
X2      0
X3      0
       ..
X380    0
X382    0
X383    0
X384    0
X385    0
Length: 377, dtype: int64

In [32]:
data_cat = df.select_dtypes(include = 'object')
data_cat = data_cat.columns.tolist()


In [33]:
for i in data_cat:
  print(f'{i}',df[i].unique(), '\n')

X0 ['k' 'az' 't' 'al' 'o' 'w' 'j' 'h' 's' 'n' 'ay' 'f' 'x' 'y' 'aj' 'ak' 'am'
 'z' 'q' 'at' 'ap' 'v' 'af' 'a' 'e' 'ai' 'd' 'aq' 'c' 'aa' 'ba' 'as' 'i'
 'r' 'b' 'ax' 'bc' 'u' 'ad' 'au' 'm' 'l' 'aw' 'ao' 'ac' 'g' 'ab'] 

X1 ['v' 't' 'w' 'b' 'r' 'l' 's' 'aa' 'c' 'a' 'e' 'h' 'z' 'j' 'o' 'u' 'p' 'n'
 'i' 'y' 'd' 'f' 'm' 'k' 'g' 'q' 'ab'] 

X2 ['at' 'av' 'n' 'e' 'as' 'aq' 'r' 'ai' 'ak' 'm' 'a' 'k' 'ae' 's' 'f' 'd'
 'ag' 'ay' 'ac' 'ap' 'g' 'i' 'aw' 'y' 'b' 'ao' 'al' 'h' 'x' 'au' 't' 'an'
 'z' 'ah' 'p' 'am' 'j' 'q' 'af' 'l' 'aa' 'c' 'o' 'ar'] 

X3 ['a' 'e' 'c' 'f' 'd' 'b' 'g'] 

X4 ['d' 'b' 'c' 'a'] 

X5 ['u' 'y' 'x' 'h' 'g' 'f' 'j' 'i' 'd' 'c' 'af' 'ag' 'ab' 'ac' 'ad' 'ae'
 'ah' 'l' 'k' 'n' 'm' 'p' 'q' 's' 'r' 'v' 'w' 'o' 'aa'] 

X6 ['j' 'l' 'd' 'h' 'i' 'a' 'g' 'c' 'k' 'e' 'f' 'b'] 

X8 ['o' 'x' 'e' 'n' 's' 'a' 'h' 'p' 'm' 'k' 'd' 'i' 'v' 'j' 'b' 'q' 'w' 'g'
 'y' 'l' 'f' 'u' 'r' 't' 'c'] 



In [34]:
def one_hot(df,variable,top_label):
    for label in top_label:
        df[variable+'_'+label] = np.where(df[variable]==label, 1,0)
        

In [35]:
########for X0
top_10 = [x for x in df.X0.value_counts().sort_values(ascending=False).head(10).index]
one_hot(df,'X0',top_10)
                                                                            

In [36]:
df.shape

(4209, 387)

In [37]:
df = df.drop(['X0'], axis=1)

# Encoding for Categorical columns

In [38]:
########for X1#########
top_10 = [x for x in df.X1.value_counts().sort_values(ascending=False).head(10).index]
one_hot(df,'X1',top_10)
                        

In [39]:
df = df.drop(['X1'], axis=1)

In [40]:
########for X2 #########
top_10 = [x for x in df.X2.value_counts().sort_values(ascending=False).head(10).index]
one_hot(df,'X2',top_10)
df = df.drop(['X2'], axis=1)

In [41]:
########for X5 #########
top_10 = [x for x in df.X5.value_counts().sort_values(ascending=False).head(10).index]
one_hot(df,'X5',top_10)
df = df.drop(['X5'], axis=1)

In [42]:
########for X6 #########
top_10 = [x for x in df.X6.value_counts().sort_values(ascending=False).head(10).index]
one_hot(df,'X6',top_10)
df = df.drop(['X6'], axis=1)

In [43]:
########for X8 #########
top_10 = [x for x in df.X8.value_counts().sort_values(ascending=False).head(10).index]
one_hot(df,'X8',top_10)
df = df.drop(['X8'], axis=1)

In [44]:
dummies1 = pd.get_dummies(df["X3"])
df = pd.concat([df,dummies1], axis=1)
df = df.drop('X3',axis=1)

In [45]:
dummies1 = pd.get_dummies(df["X4"])
df = pd.concat([df,dummies1], axis=1)
df = df.drop('X4',axis=1)

# Building Pipelines

In [46]:
X = df.drop(columns=['y'])
y = df.iloc[:,0]

from sklearn.model_selection import train_test_split #import the required function
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 5)

In [47]:
pipe = Pipeline([('feature',SelectKBest(mutual_info_regression, k = 25)),('scaler', MinMaxScaler()), ('xgb', xgb.XGBRegressor(learning_rate =1 , n_estimators=100, verbosity = 0))])


In [48]:
pipe.fit(X_train, y_train)


Pipeline(steps=[('feature',
                 SelectKBest(k=25,
                             score_func=<function mutual_info_regression at 0x0000021653457820>)),
                ('scaler', MinMaxScaler()),
                ('xgb',
                 XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              gamma=0, gpu_id=-1, grow_policy='depthwise',
                              importance_type=None, interaction_constraints='',
                              learning_rate=1, max_bin=256, max_cat_to_onehot=4,
                              max_delta_step=0, max_depth=6, max_leaves=0,
                              min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=0, num_parallel_tree=1, predictor='auto',
                              random_state=0, reg_alpha=0, reg_lambda=1, ...))])

In [49]:
pipe.score(X_test, y_test)

0.605102329200379